In [122]:
import numpy as np
from scipy import optimize

In [199]:
# Objective Function: 50x_1 + 80x_2
# Constraint 1: 5x_1 + 2x_2 <= 20
# Constraint 2: -10x_1 + -12x_2 <= -90

result = optimize.linprog(
    c = [50, 80],  # Cost function: 50x_1 + 80x_2
    A_ub = [[5, 2], 
          [1, 0],
          [-1, 0],
          [0, 1],
          [0, -1],
         ],  # Coefficients for inequalities
    b_ub = [20, 
          10,
          0,
          5,
          0
         ],  # Constraints for inequalities: 20 and -90
    bounds=(0, None), # Bounds on x, 0 <= x_i <= +oo by default
)

print(result)
50*result.x[0] + 80*result.x[1]

     con: array([], dtype=float64)
     fun: 2.426608723668483e-09
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([2.00000000e+01, 1.00000000e+01, 4.24974919e-11, 5.00000000e+00,
       3.77167660e-12])
  status: 0
 success: True
       x: array([4.24974919e-11, 3.77167660e-12])


2.426608723668483e-09

In [200]:
#TODO faire tous les cas (si il n'y a pas de windturbine ou pas de kerosine plant
import json

with open('payload4.json', 'r') as f:
    data = json.load(f)


load = data['load']

fuels = data['fuels']
#cost_gas = fuels['gas(euro/MWh)']
#cost_kerosine = fuels['kerosine(euro/MWh)']
#cost_co2 = fuels['co2(euro/ton)']
#cost_wind = fuels['wind(%)']

powerplants = data['powerplants']
nplants = len(powerplants)

c = []*nplants
A_ub = [] # 2 "load" conditions and 2 boundary condition for each powerplant (pmin and pmax)
b_ub =[]
bounds=(0, None)



#load conditions
# (sum of p_i) <= load
A_ub.append([1]*nplants)
b_ub.append(load)

# load <= (sum of p_i) ( -(sum of p_i) <= -load )
A_ub.append([-1]*nplants)
b_ub.append(-load)



for i, powerplant in enumerate(powerplants):
    print(powerplant)
    if powerplant['type'] == 'gasfired':
        c.append(fuels['gas(euro/MWh)']/powerplant["efficiency"])
        
        pminConstraintA = [0]*nplants
        pminConstraintA[i] = -1
        pminConstraintb =  -powerplant['pmin']
        
        pmaxConstraintA = [0]*nplants
        pmaxConstraintA[i] = 1
        pmaxConstraintb =  powerplant['pmax']
        
        A_ub.append(pminConstraintA)
        A_ub.append(pmaxConstraintA)
        b_ub.append(pminConstraintb)
        b_ub.append(pmaxConstraintb)

        
    if powerplant['type'] == 'windturbine':
        c.append(0) #wind is free.
        
        pminConstraintA = [0]*nplants
        pminConstraintA[i] = -1
        pminConstraintb =  -powerplant['pmin']*fuels['wind(%)']/100 
        
        pmaxConstraintA = [0]*nplants
        pmaxConstraintA[i] = 1
        pmaxConstraintb =  powerplant['pmax']*fuels['wind(%)']/100
        
        A_ub.append(pminConstraintA)
        A_ub.append(pmaxConstraintA)
        b_ub.append(pminConstraintb)
        b_ub.append(pmaxConstraintb)
        #ne pas oublier de mettre faire rentrer le vent dans les boundaries
        
    if powerplant['type'] == 'turbojet':
        c.append(fuels['kerosine(euro/MWh)']/powerplant["efficiency"])
        
        pminConstraintA = [0]*nplants
        pminConstraintA[i] = -1
        pminConstraintb =  -powerplant['pmin']
        
        pmaxConstraintA = [0]*nplants
        pmaxConstraintA[i] = 1
        pmaxConstraintb =  powerplant['pmax']
        
        A_ub.append(pminConstraintA)
        A_ub.append(pmaxConstraintA)
        b_ub.append(pminConstraintb)
        b_ub.append(pmaxConstraintb)



{'name': 'windpark1', 'type': 'windturbine', 'efficiency': 1, 'pmin': 0, 'pmax': 3000}
{'name': 'gasfiredbig1', 'type': 'gasfired', 'efficiency': 0.53, 'pmin': 100, 'pmax': 460}


In [201]:
print(np.array(A_ub))
print(np.array(b_ub))
print(c)

[[ 1  1]
 [-1 -1]
 [-1  0]
 [ 1  0]
 [ 0 -1]
 [ 0  1]]
[ 110. -110.    0. 1800. -100.  460.]
[0, 25.28301886792453]


In [202]:
solution = optimize.linprog(c = c,
                          A_ub = A_ub,
                          b_ub = b_ub,
                          bounds = bounds)['x']
solution = np.around(solution, decimals = 1)

In [203]:
print(solution)
print(c)
np.dot(solution,c)

[ 10. 100.]
[0, 25.28301886792453]


2528.301886792453

In [204]:
result = []
for i, powerplant in enumerate(powerplants):
    result.append({'name':powerplant['name'], 'p': solution[i]})

In [205]:
result

[{'name': 'windpark1', 'p': 10.0}, {'name': 'gasfiredbig1', 'p': 100.0}]